In [1]:
import os

import numpy as np

from sklearn.preprocessing import OneHotEncoder

# Load data from each folder in /data/toPreProc/

data_dict = {}
label_dict = {}

for folder in os.listdir('data/toPreProc/'):
    print("Folder name: ", folder)
    data_dict[folder] = np.load(f'data/toPreProc/{folder}/{folder}PreprocessedData.npy')
    labels = np.load(f'data/toPreProc/{folder}/{folder}Labels.npy', allow_pickle=True)
    label_dict[folder] = labels[:-1, 2]
    print("Data shape: ", data_dict[folder].shape)
    print("Label shape: ", label_dict[folder].shape)

# One hot encode the labels
ohes = {}
for key, y in label_dict.items():
    ohes[key] = OneHotEncoder(handle_unknown='ignore', sparse_output=False).fit(y)
    label_dict[key] = ohes[key].transform(y)
    print(label_dict[key])

Folder name:  2023-12-18_003


FileNotFoundError: [Errno 2] No such file or directory: 'data/toPreProc/2023-12-18_003/2023-12-18_003PreprocessedData.npy'